In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree


In [2]:
from sqlalchemy import create_engine
from hold import connection_string

In [3]:
engine = create_engine(f'{connection_string}', encoding='iso-8859-1', connect_args={'connect_timeout': 10})

In [4]:
gtdDF = pd.read_sql_table('global_terrorism', con= engine)

In [5]:
casualties = gtdDF['nkill'] + gtdDF['nwound']
casualties.sum()

554384.0

In [6]:
gtdDF['casualties'] = gtdDF['nkill'] + gtdDF['nwound']
gtdDF.head()

,iyear,imonth,iday,date,country_txt,region_txt,provstate,city,latitude,longitude,...,weaptype1_txt,weapsubtype1_txt,weaptype2_txt,weapsubtype2_txt,kidhijcountry,ransom,nkill,nwound,index1,casualties
0,2000,1,1,2000-01-01,Namibia,Sub-Saharan Africa,Kavango,Muitjiku,-17.91,19.99,...,Firearms,Unknown Gun Type,None,None,None,NaN,0.0,7.0,1.0,7.0
1,2000,1,1,2000-01-01,Namibia,Sub-Saharan Africa,Kavango,Muitjiku,-17.91,19.99,...,Firearms,Unknown Gun Type,None,None,None,0.0,0.0,7.0,2.0,7.0
2,2000,1,1,2000-01-01,India,South Asia,Jammu and Kashmir,Udhampur,32.93,75.14,...,Firearms,Unknown Gun Type,None,None,None,NaN,1.0,2.0,3.0,3.0
3,2000,1,1,2000-01-01,Kosovo,Eastern Europe,Kosovo (Province),Peje,42.66,20.31,...,Explosives/Bombs/Dynamite,Unknown Explosive Type,None,None,None,NaN,0.0,1.0,4.0,1.0
4,2000,1,1,2000-01-01,Somalia,Sub-Saharan Africa,Banaadir,Mogadishu,2.11,45.39,...,Firearms,Unknown Gun Type,None,None,None,NaN,6.0,10.0,5.0,16.0


In [7]:
RegionCasualtiesDF = gtdDF[['index1','region_txt','casualties']]
RegionCasualtiesDF.head()

,index1,region_txt,casualties
0,1.0,Sub-Saharan Africa,7.0
1,2.0,Sub-Saharan Africa,7.0
2,3.0,South Asia,3.0
3,4.0,Eastern Europe,1.0
4,5.0,Sub-Saharan Africa,16.0


In [8]:
RegionCasualties2DF = pd.get_dummies(RegionCasualtiesDF)
RegionCasualties2DF.head()

,index1,casualties,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe
0,1.0,7.0,0,0,0,0,0,0,0,0,0,0,1,0
1,2.0,7.0,0,0,0,0,0,0,0,0,0,0,1,0
2,3.0,3.0,0,0,0,0,0,0,0,0,1,0,0,0
3,4.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0
4,5.0,16.0,0,0,0,0,0,0,0,0,0,0,1,0


In [9]:
RegionCasualtiesMergeDF = result = pd.merge(RegionCasualties2DF,RegionCasualtiesDF[['index1','region_txt']],on='index1')
#RegionCasualtiesMergeDF
RegionCasualtiesMergeDF.head()

,index1,casualties,region_txt_Australasia & Oceania,region_txt_Central America & Caribbean,region_txt_Central Asia,region_txt_East Asia,region_txt_Eastern Europe,region_txt_Middle East & North Africa,region_txt_North America,region_txt_South America,region_txt_South Asia,region_txt_Southeast Asia,region_txt_Sub-Saharan Africa,region_txt_Western Europe,region_txt
0,1.0,7.0,0,0,0,0,0,0,0,0,0,0,1,0,Sub-Saharan Africa
1,2.0,7.0,0,0,0,0,0,0,0,0,0,0,1,0,Sub-Saharan Africa
2,3.0,3.0,0,0,0,0,0,0,0,0,1,0,0,0,South Asia
3,4.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,Eastern Europe
4,5.0,16.0,0,0,0,0,0,0,0,0,0,0,1,0,Sub-Saharan Africa


In [10]:
# Assign X (data) and y (target)
data = RegionCasualtiesMergeDF.drop(["index1","casualties","region_txt"], axis=1)
target = RegionCasualtiesMergeDF["region_txt"]
feature_names = data.columns

print(data.shape, target.shape)

(92700, 12) (92700,)


In [11]:
# Split the data into training and testing
### BEGIN SOLUTION
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
### END SOLUTION

In [12]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

1.0

In [14]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.3060479887474645, 'region_txt_Middle East & North Africa'),
 (0.2891307968364563, 'region_txt_South Asia'),
 (0.12282112243959845, 'region_txt_Sub-Saharan Africa'),
 (0.11904769539534175, 'region_txt_Southeast Asia'),
 (0.06284232962560984, 'region_txt_Eastern Europe'),
 (0.0476714306437956, 'region_txt_Western Europe'),
 (0.03305813567376372, 'region_txt_South America'),
 (0.009181987920520171, 'region_txt_North America'),
 (0.004013934017585076, 'region_txt_Central Asia'),
 (0.003358829550527919, 'region_txt_East Asia'),
 (0.0015773134506350082, 'region_txt_Central America & Caribbean'),
 (0.0012484356987015506, 'region_txt_Australasia & Oceania')]